In [1]:
%load_ext autoreload
%autoreload 2

In [2]:

import os
import sys
import json

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import font_manager
%matplotlib inline
import matplotlib.colors as colors


In [3]:
# resource configuration
rcParams['figure.dpi'] = 200

# fount information
rcParams['font.family'] = 'Equinor'
rcParams['font.size'] = 9

# fornt location
font_path = r'Equinor_regular'
font_files = font_manager.findSystemFonts(fontpaths=font_path)

for font_file in font_files:
    font_manager.fontManager.addfont(font_file)

In [4]:
# where WellClass and Ga[ codes are located
sys.path.append('../src')

In [5]:
# WellClass
# from well_class_v2 import csv_parser, Well
from WellClass.libs.utils import (
    csv_parser,
    yaml_parser,
)
from WellClass.libs.well_class import Well
from WellClass.libs.well_pressure import Pressure

## Some user options

In [6]:
# pick an example from given three options

case_type = 'cosmo'

case_type = 'smeaheia_v1'

case_type = 'smeaheia_v2'

In [7]:
# where pvt located
pvt_path = '../test_data/pvt_constants/'

# Examples

The following are the test examples.

In [8]:
# examples
smeaheia_v1 = {'well_input': r'GaP_input_Smeaheia_v3.csv', 
               'well_input_yaml': r'smeaheia.yaml', 
            #    'sim_path': r'/scratch/SCS/eim/SMEAHEIA', 
               'sim_path': r'../test_data/examples/smeaheia_v1',
               'simcase': r'GEN_NOLGR_PH2'}
smeaheia_v2 = {'well_input': r'GaP_input_Smeaheia_v3.csv', 
               'well_input_yaml': r'smeaheia.yaml', 
            #    'sim_path': r'/scratch/SCS/bkh/wbook/realization-0/iter-0/pflotran/model', 
               'sim_path': r'../test_data/examples/smeaheia_v2', 
               'simcase': r'TEMP-0'}
cosmo = {
         'well_input': r'GaP_input_Cosmo_v3.csv', 
         'well_input_yaml': r'cosmo.yaml', 
        #  'sim_path': r'/scratch/SCS/bkh/well_class_test1/realization-0/iter-0/pflotran/model', 
         'sim_path': r'../test_data/examples/cosmo', 
         'simcase': r'TEMP-0'}

In [9]:
examples = {
    'smeaheia_v1': smeaheia_v1,
    'smeaheia_v2': smeaheia_v2,
    'cosmo': cosmo
}

In [10]:
# the selected example for testing
case = examples[case_type]


In [11]:

# root_path = '/scratch/SCS/gpb/SCREEN/GaP_code'

sim_path = case['sim_path']


# Load well CSV file

Process CSV with well class.
Predefine a dictionary that includes the input CSV well file, the simulation path, and the PFT sim case name

In [12]:
# where well configuration file is located
well_name = os.path.join(sim_path, case['well_input'])

# load the well information
well_csv = csv_parser(well_name)

In [13]:
#Process well by running well class
my_well = Well( header       = well_csv['well_header'], 
                drilling     = well_csv['drilling'],
                casings      = well_csv['casing_cement'],
                geology      = well_csv['geology'],
                barriers     = well_csv['barriers'], 
                barrier_perm = well_csv['barrier_permeability'],
                co2_datum    = well_csv['co2_datum'],
           )

In [14]:
aa = pd.DataFrame(my_well.geology)
aa

,top_rkb,geol_unit,reservoir_flag,top_msl,base_msl
0,336,NORDLAND GP,False,312.5,511.5
1,535,ROGALAND GP,False,511.5,822.5
2,846,SHETLAND GP,False,822.5,1057.5
3,1081,CROMER KNOLL GP,False,1057.5,1085.5
4,1109,DRAUPNE FM,False,1085.5,1192.5
5,1216,HEATHER FM,False,1192.5,1214.5
6,1238,SOGNEFJORD FM,True,1214.5,1282.5
7,1306,HEATHER FM,False,1282.5,1342.5
8,1366,FENSFJORD FM,True,1342.5,1571.5
9,1595,HEATHER FM,False,1571.5,1574.5


In [15]:
my_well.co2_datum

1282.5

In [16]:
well_csv['reservoir_pressure']

{'depth_msl': 1217.0, 'RP1': nan, 'RP2': 91.0, 'RP3': 20.0}

In [17]:
my_pressure = Pressure(
    header      = well_csv['well_header'],
    reservoir_P = well_csv['reservoir_pressure'],
    co2_datum   = well_csv['co2_datum'],
    pvt_path    = pvt_path,
)

Hydrostatic pressure at reference depth 1217 is 120.43
RP1 set as hydrostatic P = 120.43 bar
RP2 is set as delta pressure, which yields P = 120.43 +91.00 = 211.43 bar
RP3 is set as delta pressure, which yields P = 120.43 +20.00 = 140.43 bar
Reference depth: 1217.0


# Load well Yaml file

Process yaml with well class.
Predefine a dictionary that includes the input yaml well file, the simulation path, and the PFT sim case name

In [18]:
# where well configuration file is located
well_name = os.path.join(sim_path, case['well_input_yaml'])

# # pydantic model
well_model = yaml_parser(well_name)
well_csv = json.loads(well_model.spec.model_dump_json())

In [19]:
#Process well by running well class
my_well_yaml = Well( header       = well_csv['well_header'], 
                     drilling     = well_csv['drilling'],
                     casings      = well_csv['casing_cement'],
                     geology      = well_csv['geology'],
                     barriers     = well_csv['barriers'], 
                     barrier_perm = well_csv['barrier_permeability'],
                     co2_datum    = well_csv['co2_datum'],
           )

In [20]:
bb = pd.DataFrame(my_well_yaml.geology)
bb

,top_rkb,geol_unit,reservoir_flag,top_msl,base_msl
0,336.0,NORDLAND GP,False,312.5,511.5
1,535.0,ROGALAND GP,False,511.5,822.5
2,846.0,SHETLAND GP,False,822.5,1057.5
3,1081.0,CROMER KNOLL GP,False,1057.5,1085.5
4,1109.0,DRAUPNE FM,False,1085.5,1192.5
5,1216.0,HEATHER FM,False,1192.5,1214.5
6,1238.0,SOGNEFJORD FM,True,1214.5,1282.5
7,1306.0,HEATHER FM,False,1282.5,1342.5
8,1366.0,FENSFJORD FM,True,1342.5,1571.5
9,1595.0,HEATHER FM,False,1571.5,1574.5


In [21]:
my_well_yaml.co2_datum

1282.5

In [22]:
well_csv['reservoir_pressure']

{'depth_msl': 1217.0, 'RP1': None, 'RP2': '91', 'RP3': '20'}

In [23]:
my_pressure_yaml = Pressure(
    header      = well_csv['well_header'],
    reservoir_P = well_csv['reservoir_pressure'],
    co2_datum   = well_csv['co2_datum'],
    pvt_path    = pvt_path,
)

Hydrostatic pressure at reference depth 1217 is 120.43
RP1 set as hydrostatic P = 120.43 bar
RP2 is set as delta pressure, which yields P = 120.43 +91.00 = 211.43 bar
RP3 is set as delta pressure, which yields P = 120.43 +20.00 = 140.43 bar
Reference depth: 1217.0
